# Exercise 2
Add a counter to `dfs()`, `bfs()`, and `astar()` to see how many states each
searches through for the same maze. Find the counts for 100 different mazes to
get statistically significant results.

## Create 100 mazes

In [27]:
from typing import List
from maze import Maze
mazes: List[Maze] = list()
for i in range(100):
    mazes.append(Maze())

In [28]:
print(mazes[10])

SXXX      
          
       X  
 X  X   X 
      X  X
    X  X X
 X X      
X         
          
        XG



## Searches with counter

In [31]:
from typing import TypeVar, Iterable, Sequence, Generic, List, Callable, Set, Deque, Dict, Any, Optional

T = TypeVar('T')

from generic_search import Node, Stack, Queue, PriorityQueue
def dfs_counter(initial: T, goal_test: Callable[[T], bool], successors: Callable[[T], List[T]]) -> (Optional[Node[T]], int):
    # frontier is where we've yet to go
    frontier: Stack[Node[T]] = Stack()
    frontier.push(Node(initial, None))
    # explored is where we've been
    explored: Set[T] = {initial}

    # keep going while there is more to explore
    while not frontier.empty:
        current_node: Node[T] = frontier.pop()
        current_state: T = current_node.state
        # if we found the goal, we're done
        if goal_test(current_state):
            return current_node, len(explored)
        # check where we can go next and haven't explored
        for child in successors(current_state):
            if child in explored:  # skip children we already explored
                continue
            explored.add(child)
            frontier.push(Node(child, current_node))
    return None, len(explored)  # went through everything and never found goal

def bfs_counter(initial: T, goal_test: Callable[[T], bool], successors: Callable[[T], List[T]]) -> (Optional[Node[T]], int):
    # frontier is where we've yet to go
    frontier: Queue[Node[T]] = Queue()
    frontier.push(Node(initial, None))
    # explored is where we've been
    explored: Set[T] = {initial}

    # keep going while there is more to explore
    while not frontier.empty:
        current_node: Node[T] = frontier.pop()
        current_state: T = current_node.state
        # if we found the goal, we're done
        if goal_test(current_state):
            return current_node, len(explored)
        # check where we can go next and haven't explored
        for child in successors(current_state):
            if child in explored:  # skip children we already explored
                continue
            explored.add(child)
            frontier.push(Node(child, current_node))
    return None, len(explored)  # went through everything and never found goal

def astar_counter(initial: T, goal_test: Callable[[T], bool], successors: Callable[[T], List[T]], heuristic: Callable[[T], float]) -> (Optional[Node[T]], int):
    # frontier is where we've yet to go
    frontier: PriorityQueue[Node[T]] = PriorityQueue()
    frontier.push(Node(initial, None, 0.0, heuristic(initial)))
    # explored is where we've been
    explored: Dict[T, float] = {initial: 0.0}

    # keep going while there is more to explore
    while not frontier.empty:
        current_node: Node[T] = frontier.pop()
        current_state: T = current_node.state
        # if we found the goal, we're done
        if goal_test(current_state):
            return current_node, len(explored)
        # check where we can go next and haven't explored
        for child in successors(current_state):
            new_cost: float = current_node.cost + 1  # 1 assumes a grid, need a cost function for more sophisticated apps

            if child not in explored or explored[child] > new_cost:
                explored[child] = new_cost
                frontier.push(Node(child, current_node, new_cost, heuristic(child)))
    return None, len(explored)  # went through everything and never found goal

## Search test

In [48]:
from maze import manhattan_distance
total_steps_dfs: int = 0
total_steps_bfs: int = 0
total_steps_astar : int = 0
    
for m in mazes:
    _, steps_dfs = dfs_counter(m.start, m.goal_test, m.successors)
    total_steps_dfs += steps_dfs
    
    _, steps_bfs = bfs_counter(m.start, m.goal_test, m.successors)
    total_steps_bfs += steps_bfs
    
    _, steps_astar = astar_counter(m.start, m.goal_test, m.successors, manhattan_distance(m.goal))
    total_steps_astar += steps_astar

In [49]:
total_steps_dfs/100

63.28

In [50]:
total_steps_bfs/100

70.02

In [51]:
total_steps_astar/100

57.52

BFS takes the most steps on average, but does find the optimal path. DFS finds the goal quicker, but the path is not always optimal. A* finds the optimal path with the least amount of steps.